<img src='https://user-images.githubusercontent.com/16665629/36117115-329c6220-1041-11e8-98d3-7cd1ce05503c.jpeg'
style="float:right;margin:0 10px 0px 0;width: 150px;">

# Compare observed and theoretical statistical distributions

In [ ]:
models_compare = {
                    'point_skymodel1.txt': 'meerkat_point_clean_mask-cube-image-pybdsf.lsm.html',
                    'point_skymodel2.txt': 'meerkat_point_ddfacet_mask-cube-image-pybdsf.lsm.html',
                    'point_skymodel3.txt': 'meerkat_point_lwimager_mask-cube-image-pybdsf.lsm.html',
                    'point_skymodel4.txt': 'meerkat_point_tclean_mask-cube-image-pybdsf.lsm.html',
                    'point_skymodel5.txt': 'meerkat_point_wsclean_mask-cube-image-pybdsf.lsm.html',
                 }
models_dr = {
            'meerkat_point_clean_mask-cube-image-pybdsf.lsm.html': 'meerkat_point_clean_mask-cube-residual.fits',
            'meerkat_point_ddfacet_mask-cube-image-pybdsf.lsm.html': 'meerkat_point_ddfacet_mask.cube.app.residual.fits',            
            'meerkat_point_lwimager_mask-cube-image-pybdsf.lsm.html': 'meerkat_point_lwimager_mask-cube-residual.fits',
            'meerkat_point_tclean_mask-cube-image-pybdsf.lsm.html': 'meerkat_point_tclean_mask-cube-residual.fits',
            'meerkat_point_wsclean_mask-cube-image-pybdsf.lsm.html': 'meerkat_point_wsclean_mask-cube-residual.fits',
            }
RESIDUALS = {
             'meerkat_point_clean_mask-cube-residual.fits': 'casa_noise.dirty.image.fits',
             'meerkat_point_ddfacet_mask.cube.app.residual.fits':'ddfacet_noise.cube.app.restored.fits',
             'meerkat_point_lwimager_mask-cube-residual.fits': 'lwimager_noise.dirty.fits',
             'meerkat_point_tclean_mask-cube-residual.fits': 'tclean_noise.dirty.image.fits',
             'meerkat_point_wsclean_mask-cube-residual.fits':'wsclean_noise-cube-dirty.fits',

}
#tolerance = 0.00002 # area factor around the source
#tolerance = 0.0001 # extended
tolerance = 0.000001 # point

In [ ]:
from ipywidgets import widgets, interact
from IPython.display import Javascript, display
from IPython.display import HTML

def run_all(ev):
    # Runn all cells
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,'
                       'IPython.notebook.get_selected_index()+20)'))

directory = widgets.Text(
    value='output',
    placeholder='output',
    description='work directory:',
    disabled=False
)

button = widgets.Button(description="Configure")
button.on_click(run_all)
display(button)
display(directory)

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} $( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit"
value="Click here to toggle on/off the raw code."></form>''')

In [ ]:
import glob
import combat
import aimfast
import matplotlib
import numpy as np
from plotly import tools
from tabletext import to_text
import plotly.graph_objs as go
from plotly import offline as py
from collections import OrderedDict
from scipy.stats import norm, linregress
from sklearn.metrics import r2_score, mean_squared_error
from plotly.graph_objs import Layout, Figure, XAxis, YAxis
from aimfast.aimfast import residual_image_stats, image_dynamic_range, model_dynamic_range#, get_json_dict
py.init_notebook_mode(connected=True)

## Photometric Measurements

In [ ]:
reload(aimfast)
a = aimfast.aimfast.plot_photometry(models_compare, label='point', all_sources=True, dir=directory.value)

## Astrometric Measurements

In [ ]:
b = aimfast.aimfast.plot_astrometry(models_compare, label='point', all_sources=True, dir=directory.value)

## Morphological Measurements

In [ ]:
c = aimfast.aimfast.plot_morphology(models_compare, label='point', all_sources=True, dir=directory.value)

## Spectral Index Measurements

In [ ]:
results = aimfast.aimfast.plot_spectrum(models_compare, label='point', all_sources=True, dir=directory.value)

In [ ]:
import matplotlib
import pylab as plt
from scipy.stats import norm
fig = plt.figure()
spi_bins = num_bins = 5
i = -1
j = 0
t = -1
counter = 0
flux_im = dict()
I_min_max = []
err_I_min_max = []
#========================
#MEAN = " Mean"
#MEDIAN = " Median"
#MODE = " Mode"
#STD = " Standard deviation"
#========================
global_table_data = [["Global", "Mean", "Median", "Mode", "Standard Deviation"]]
for input_model, output_model in sorted(models_compare.items()):
    header = output_model[:-9]
    bin_table_data = [[header.split('-')[-1].upper(), "Mean", "Median", "Mode", "Standard Deviation"]]
    i += 1
    t += 1
    header = 'point-model_a_{}'.format(t)
    counter+=1
    I_in = []
    name_labels = []
    spi_in_data = []
    spi_out_data = []
    spi_err_data = []
    dist_from_phase = []
    key = 'spectrum'
    num_data_points = len(results[header][key])
    for n in range(num_data_points):
        spi_out_data.append(results[header][key][n][0])
        spi_err_data.append(results[header][key][n][1])
        spi_in_data.append(results[header][key][n][2])
        dist_from_phase.append(results[header][key][n][3])
        I_in.append(results[header][key][n][4])
        name_labels.append(results[header][key][n][5])
    zipped_props = zip(I_in, spi_out_data, spi_err_data, spi_in_data, dist_from_phase, name_labels)
    (I_in, spi_out_data, spi_err_data, spi_in_data, dist_from_phase, name_labels) = zip(
        *sorted(zipped_props, key=lambda x: x[0]))
    
#===========================================================================================================
    spi_in_data_stats = []
    spi_out_data_stats = []
    for spi_in, spi_out in zip(spi_in_data, spi_out_data):
        if spi_out:
            spi_in_data_stats.append(spi_in)
            spi_out_data_stats.append(spi_out)
    spi_R_score = r2_score(spi_in_data_stats, spi_out_data_stats)
    spi_MSE = mean_squared_error(spi_in_data_stats, spi_out_data_stats)
    spi_out_in = [float(spi_out)/spi_in for spi_out,spi_in in
                      zip(spi_out_data_stats,spi_in_data_stats)]
    spi_in_data = spi_in_data_stats
    spi_out_data = spi_out_data_stats
#===========================================================================================================
    
    ranger = num_data_points/num_bins
    start, end = [-ranger, 0]
    if num_bins > 1:
        for b in range(num_bins):
            end += ranger
            start += ranger
            spi_out_data_range = spi_out_data[start:num_data_points if (b + 1) == num_bins else end],
            n, bins, patches = matplotlib.pyplot.hist(spi_out_data_range,
                                                      spi_bins, normed=False, facecolor='blue',
                                                      orientation='horizontal', alpha=0.5)
            (mu, sigma) = norm.fit(spi_out_data_range)
            median = np.median(spi_out_data_range)
            peak = [bins[i] for i,v in enumerate(n) if v == n.max()][0]
            bin_table_data.append(["BIN-{:d}".format(b+1), "{:4f}".format(mu), "{:4f}".format(median),
                                  "{:4f}".format(peak), "{:4f}".format(sigma)])

    print(to_text(bin_table_data))
    n, bins, patches = matplotlib.pyplot.hist(spi_out_data_range,
                                              spi_bins, normed=False, facecolor='blue',
                                              orientation='horizontal', alpha=0.5)
    (mu, sigma) = norm.fit(spi_out_data)
    median = np.median(spi_out_data)
    peak = [bins[i] for i,v in enumerate(n) if v == n.max()][0]
    global_table_data.append([header.split('-')[-1].upper(), "{:4f}".format(mu), "{:4f}".format(median),
                            "{:4f}".format(peak), "{:4f}".format(sigma)])
    counter += 1

plt.close(fig)
print(to_text(global_table_data))

## Dynamic Range

In [ ]:
reload(aimfast)
prefix = 'meerkat_point_'
beam_factor = 4
DRs = OrderedDict()
im_titles = []
dir = directory.value
for m, r in models_dr.items():
    DRs[r.split('.')[0].split(prefix)[-1].split('_')[0].upper()] = model_dynamic_range('{:s}/{:s}'.format(dir, m),'{:s}/{:s}'.format(dir, r), 6, beam_factor)
#    DRs[r[:-19]] = model_dynamic_range('{:s}/{:s}'.format(dir, m),'{:s}/{:s}'.format(dir, r), 94, beam_factor)
zipped_props = zip(DRs.keys(), DRs.values())
ims, values = zip(*sorted(zipped_props, key=lambda x: x[0]))
drs, names = [], []
for val in values:
    drs.append(val['global_rms'])
    #names.append("DR=({:f},{:f})".format(val[1], val[-1]))
    names.append("DR={:f}".format(val['global_rms']))
aimfast.aimfast.aimfast_plotly(X1=ims, Y1=np.array(drs), plot_title='Point Source Simulation',
                               X2=ims, Y2=np.array([100, 100, 100, 100, 100]),
                               x_title='Images', y_title='Dynamic Range')

## Residual Statistics

In [ ]:
aimfast.aimfast.print_residual_stats(sorted(RESIDUALS.keys()), prefix='meerkat_point_', suffix='residual.fits',
                                     replace='cube-residual', units='mJy', dir=directory.value)

#### Source residual-to-noise ration measurements

In [ ]:
aimfast.aimfast.plot_residuals_noise(RESIDUALS, skymodel='point_skymodel1.txt', area_factor=5.0, dir=directory.value)

#### Random source residual-to-noise ration measurements

In [ ]:
aimfast.aimfast.plot_residuals_noise(RESIDUALS, skymodel=None, area_factor=5.0, points=100, dir=directory.value)

## View Reconstructed Maps

In [ ]:
%%html
<iframe src="http://js9.si.edu" width=990 height=740></iframe>

## Source meta-data

In [ ]:
src_nm = widgets.Text(
    value='',
    placeholder='ay040',
    description='Source Name:',
    disabled=False
)
display(src_nm)

In [ ]:
def run_all(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,'
                       'IPython.notebook.get_selected_index()+2)'))

button = widgets.Button(description="Search")
button.on_click(run_all)
display(button)